In [104]:
import pandas as pd

df = pd.read_pickle('/home/kai/data/shiyi/AlphaBoosting/automl/automl_app/project1/data/train.pkl')

df.shape


(1000000, 9)

In [105]:
df.head(2)

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,label
0,7,19,2,18,21,157,133,6,0
1,4,3,1,12,18,79,172,4,0


In [121]:

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

le = LabelEncoder()
lg = LinearRegression()

group_col = 'Month'
time_col = 'DayOfWeek'
value_col = 'label'

le_result = []
lg_result = []
gp = df.head(10).groupby(by=group_col)
for i, group in gp:
    lg.fit(group[[time_col]], group[[value_col]])
    lg_result.append([i, lg.coef_[0][0]]) 
    
    temp = le.fit_transform(group[[time_col]])
    le_result.append([i, temp])

/home/kai/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [122]:
lg_result

[[1, 0.0], [2, 0.0], [3, -0.0], [4, 0.99999999999999967], [6, 0.0], [7, -0.0]]

In [141]:
for group_id, group in df.head(10).groupby(group_col):
    print("=============")
    print('id', group_id)
    x = group[[time_col]]
    y = group[[value_col]]
    print(x, y)
    lg.fit(x, y)
    print(lg.coef_[0][0])
    break

id 1
   DayOfWeek
7          5    label
7      0
0.0


In [182]:
import multiprocessing
import numpy as np
from tqdm import tqdm

def _apply_group2(args):
    func, group_id, x, y = args
    return [group_id] + func(x,y)

def apply_by_multiprocessing(func,**kwargs):
    workers = kwargs.pop('workers')
    group_list = kwargs.pop('group_list')
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_group2, [(func, group_id, x, y) for group_id, x, y in group_list])
    pool.close()
    return result

def liner_reg(x, y, **kwargs):
    lg = LinearRegression()
    lg.fit(x, y)
    return [lg.coef_[0][0], 1]

# def scaler_transform(x, **kwargs):
#     scaler_dict=kwargs.pop('scaler_dict')
#     col = kwargs.pop('col')
#     return scaler_dict[col].transform(x)[0][0]

grouplist = [(group_id, group[time_col].values) for group_id, group in df.head(10).groupby(group_col)]

grouplist

final = apply_by_multiprocessing(label_encode, workers=5, group_list=grouplist)

final

In [183]:
grouplist2 = [(group_id, group[[time_col]], group[[value_col]]) for group_id, group in df.head(10).groupby(group_col)]

In [184]:
print(grouplist2[0][0])
print(grouplist2[0][1])
print(grouplist2[0][2])

1
   DayOfWeek
7          5
   label
7      0


In [185]:
final2 = apply_by_multiprocessing(liner_reg, workers=10, group_list=grouplist2)

In [186]:
final2

[[1, 0.0, 1],
 [2, 0.0, 1],
 [3, -0.0, 1],
 [4, 0.99999999999999967, 1],
 [6, 0.0, 1],
 [7, -0.0, 1]]

for col in tqdm(numeric_features):
    train[col]=apply_by_multiprocessing(train[col], scaler_transform, workers=workers, col=col, scaler_dict=scaler_dict)
    test[col]=apply_by_multiprocessing(test[col], scaler_transform, workers=workers, col=col, scaler_dict=scaler_dict)